# Prueba técnica ecosistemas

El objetivo principal de este proyecto es implementar una herramienta automatizada que permita a la empresa BATSEJ OPEN FINANCE S.A. calcular mensualmente las comisiones de las cuentas bancarias de sus empresas contratantes de manera rápida, precisa y confiable. Esta automatización no solo optimiza el tiempo invertido en el proceso, sino que también aumenta la fiabilidad de los cálculos y mejora el rendimiento operativo.

Al reducir el esfuerzo manual y minimizar los errores humanos, la automatización garantiza una mayor eficiencia operativa, lo cual se traduce en un claro aumento del retorno sobre la inversión (ROI). Esto no solo mejora la rentabilidad del negocio, sino que también eleva el nivel de satisfacción y fidelización de los clientes, al brindarles un servicio más ágil y confiable.

Además, al implementar esta solución novedosa, BATSEJ OPEN FINANCE S.A. se posiciona de manera más competitiva en el mercado, abriendo la puerta a nuevos proyectos y desafíos. La capacidad de responder con rapidez y precisión a las necesidades de los clientes fortalece la relación comercial, mejora la reputación de la empresa y genera oportunidades para expandir su portafolio de servicios.

Para comenzar importarmos las librerías necesarias. Primero, utilizamos sqlite3, ya que nuestra base de datos está en este formato. Esta librería nos permite conectarnos directamente a la base de datos, facilitando la búsqueda, extracción y análisis de las tablas necesarias para el cálculo de comisiones.

A continuación, empleamos pandas, una librería clave de Python para la manipulación de datos. Con pandas, podemos construir y gestionar de manera eficiente diferentes data frames, lo que nos permitirá analizar grandes volúmenes de información de forma estructurada.

Para manejar los archivos en formato Excel, hacemos uso de openpyxl, que facilita tanto la creación como la edición de estos archivos. Esto nos permitirá exportar los resultados del análisis de comisiones en un formato fácil de compartir y presentar.

Finalmente, utilizamos las librerías smtplib y email.message para automatizar el envío de correos electrónicos. Estas herramientas nos permiten construir un script que envía automáticamente los resultados obtenidos, generando un flujo continuo entre el análisis y la distribución de la información. Así, aseguramos que las empresas eciban la información sobre sus comisiones de manera oportuna y precisa.



In [ ]:
# Importamos las librerias requeridas para la implementación del ejercicio

import sqlite3
import pandas as pd
import openpyxl
import smtplib
from email.message import EmailMessage

Ahora, implementamos la clase Calculo_comisiones, la cual se diseñó con el propósito de definir las funciones clave que automatizan el cálculo y el cobro de comisiones, cumpliendo con los requerimientos establecidos. A continuación, se explican las características principales de cada función:

- Función obtener_datos: Esta función establece la conexión a la base de datos SQlite permitiendo extraer la información requerida desde las tablas "apicall" y "commerce". El siguiente paso es unificar ambas tablas haciendo uso del lenguaje SQL, en este caso usamos la instrucción left join tomando la tabla apicall como la principal (tabla izquierda) para unir en cada registro los campos commerce_nit, commerce_name, commerce_status y commerce_email de la tabla commerce, se usa como llave de cruce el campo commerce_id. Además,  Además, se filtran los datos según el mes de interés mediante la instrucción WHERE, garantizando que solo se extraiga la información correspondiente al período de cálculo.

- Función cobro_peticiones: Esta función automatiza el cálculo de las comisiones para cada empresa, para ello se requiere ingresar como hiperparametros los campos referentes al número total de peticiones, tanto exitosas como fallidas, así como el IVA y precios referetes a la información suministrada. Cabe resaltar que las condiciones de contrato dadas por las empresas son las que se generan en esta función, se automatizan los campos de tal forma que si se requiere hacer cambios futuros se ingresen dichos cambios a la función sin modificar la estructura del programa. Este diseño flexible asegura que cualquier cambio en los parámetros pueda ser fácilmente integrado al programa, cumpliendo así el objetivo de automatización establecido desde el principio.

- Función calcular_comision:Aquí se genera la tabla final llamada resultado_tab, que consolida para cada empresa el número total de peticiones (exitosas y fallidas), junto con su NIT, nombre y correo electrónico. Esto se logra mediante un cruce (merge) entre la tabla de resultados y la tabla commerce, nuevamente utilizando el campo commerce_id como llave. Con estos datos, se llama a la función cobro_peticiones definida anteriormente, para calcular automáticamente el valor_total, valor_comisión y valor_iva, que constituyen los montos finales a cobrar. Esta tabla final es fundamental para enviar la información consolidada a los clientes.

- Función enviar_correo: Esta función automatiza el envío de las facturas generadas a las empresas contratantes. Cada factura contiene un contenido detallado de los montos calculados. Para ejecutar esta función, se requieren como parámetros el archivo Excel con los detalles de la comisión, el destinatario y remitente, junto con las credenciales de correo (cuenta y contraseña). En este ejercicio, se simula el envío con un correo ficticio, pero puede reemplazarse con uno real para validar la operación de manera efectiva

In [4]:
class Calculo_comisiones:
    
    def __init__(self, conn):
        self.conn = conn
        self.cursor = self.conn.cursor()

    def obtener_datos(self, mes):
        query = f"""
        CREATE TABLE IF NOT EXISTS resultado AS
        SELECT A.*, C.* 
        FROM apicall AS A
        LEFT JOIN commerce AS C
        ON A.commerce_id = C.commerce_id
        WHERE strftime('%Y-%m', A.date_api_call) = '{mes}'
        """
        self.cursor.execute(query)
        self.conn.commit()

    def cobro_peticiones(self, empresa, peticiones_exitosas, peticiones_fallidas, peticiones_totales, precios, iva):
        total = 0
        if empresa == 'Innovexa Solutions':
            total = peticiones_exitosas * precios['Innovexa Solutions'] + peticiones_exitosas * precios['Innovexa Solutions'] * iva
        elif empresa == 'NexaTech Industries':
            if peticiones_totales <= 10000:
                total = peticiones_exitosas * precios['NexaTech Industries'][0] + peticiones_exitosas * precios['NexaTech Industries'][0] * iva
            elif peticiones_totales <= 20000:
                total = peticiones_exitosas * precios['NexaTech Industries'][1] + peticiones_exitosas * precios['NexaTech Industries'][1] * iva
            else:
                total = peticiones_exitosas * precios['NexaTech Industries'][2] + peticiones_exitosas * precios['NexaTech Industries'][2] * iva
        elif empresa == 'QuantumLeap Inc.':
            total = peticiones_exitosas * precios['QuantumLeap Inc.'] + peticiones_exitosas * precios['QuantumLeap Inc.'] * iva
        elif empresa == 'Zenith Corp.':
            if peticiones_totales >= 0 and peticiones_totales <= 22000:
                total = peticiones_exitosas * precios['Zenith Corp.'][0] + peticiones_exitosas * precios['Zenith Corp.'][0] * iva
            elif peticiones_totales >= 22001:
                total = peticiones_exitosas * precios['Zenith Corp.'][1] + peticiones_exitosas * precios['Zenith Corp.'][1] * iva
            if peticiones_fallidas >= 6000:
                total = total - total * 0.05
        elif empresa == 'FusionWave Enterprises':
            total = peticiones_exitosas * precios['FusionWave Enterprises'] + peticiones_exitosas * precios['FusionWave Enterprises'] * iva
            if 2500 <= peticiones_fallidas <= 4500:
                total = total - total * 0.05  # Descuento del 5% antes de IVA
            elif peticiones_fallidas > 4501:
                total = total - total * 0.08
        return total

    def calcular_comision(self, mes, precios, iva):
        self.obtener_datos(mes)
        
        resultado = pd.read_sql("SELECT * FROM resultado", self.conn)
        commerce = pd.read_sql("SELECT * FROM commerce", self.conn)
        
        resultado_tab = pd.crosstab(resultado['commerce_id'], resultado['ask_status'])
        resultado_tab['Total'] = resultado_tab['Successful'] + resultado_tab['Unsuccessful']

        resultado_tab = resultado_tab.merge(commerce[['commerce_id', 'commerce_name', 'commerce_nit', 'commerce_email']], 
                                            left_on='commerce_id', right_on='commerce_id', how='left')
        
        resultado_tab['Cobro Total'] = resultado_tab.apply(
            lambda row: self.cobro_peticiones(row['commerce_name'], row['Successful'], row['Unsuccessful'], row['Total'], precios, iva), axis=1)

        resultado_tab['Fecha-Mes'] = mes
        resultado_tab['Valor_comision'] = resultado_tab['Cobro Total'] / (1 + iva)
        resultado_tab['Valor_iva'] = resultado_tab['Valor_comision'] * iva
        resultado_tab['Valor_Total'] = resultado_tab['Cobro Total']
        
        return resultado_tab
    
    def enviar_correo(self, archivo_excel, destinatario, remitente, contrasena):
        # Crear el mensaje
        msg = EmailMessage()
        msg.set_content("Hola,\n\nAdjunto se encuentra el archivo con los resultados de las comisiones calculadas.\n\nSaludos,\n equipo BATSEJ OPEN FINANCE S.A")
        msg['Subject'] = 'Resultados de Comisiones'
        msg['From'] = remitente
        msg['To'] = destinatario

        # Adjuntar el archivo Excel
        with open(archivo_excel, 'rb') as f:
            file_data = f.read()
            file_name = f.name

        msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)

        # Conectar al servidor SMTP de Outlook y enviar el mensaje
        smtp_server = 'smtp.office365.com'
        smtp_port = 587

        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()  # Establecer una conexión segura
                server.login(remitente, contrasena)
                server.send_message(msg)
                print("Correo electrónico enviado exitosamente.")
        except Exception as e:
            print(f"Error al enviar el correo: {e}")


Con las funciones de la clase Calculo_comisiones definidas, procedemos a su ejecución. La creación de una clase con funciones parametrizadas nos permite ajustar los parámetros de manera flexible. Esto significa que, ante cambios en valores como la fecha de interés, el costo por petición exitosa o el IVA, solo será necesario ingresar los nuevos valores 
<!-- sin recalcular manualmente los campos de interés. -->

Este enfoque automatizado reduce significativamente la necesidad de realizar cálculos manuales, liberando tiempo para que el personal pueda enfocarse en otras tareas. Además, se mitiga el riesgo de errores humanos, ya que los resultados son consistentes y estandarizados, eliminando la variabilidad que puede surgir en los cálculos manuales.

In [5]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('C:/Prueba_ecosistemas/database.sqlite')

# Crear instancia de la clase
calculo = Calculo_comisiones(conn)

In [8]:
precios = {
    'Innovexa Solutions': 300,
    'NexaTech Industries': [250,  # Asignar si hay entre 0 – 10.000 peticiones totales
                            200,  # Asignar si hay entre 10.001 – 20.000 peticiones totales
                            170], # Asignar si hay más de 20.001 peticiones totales
    'QuantumLeap Inc.': 600, 
    'Zenith Corp.': [250,  # Asignar si hay entre 0 – 22.000 peticiones totales
                     130], # Asignar si hay más de 22.001 peticiones totales
    'FusionWave Enterprises': 300
}
iva = 0.19

In [10]:
# Calcular comisiones para un mes y año dados
mes = '2024-07'
resultado_tab = calculo.calcular_comision(mes, precios, iva)

In [11]:
resultado_tab

,commerce_id,Successful,Unsuccessful,Total,commerce_name,commerce_nit,commerce_email,Cobro Total,Fecha-Mes,Valor_comision,Valor_iva,Valor_Total
0,3VYd-4lzT-mTC3-DQN5,240591,59921,300512,Zenith Corp.,28960112,zenithcorp.@gemaily.net,3.535846e+07,2024-07,29712988.5,5.645468e+06,3.535846e+07
1,GdEQ-MGb7-LXHa-y6cd,239507,60145,299652,FusionWave Enterprises,919341007,fusionwaveenterprises@microfitsof.com,7.866368e+07,2024-07,66103932.0,1.255975e+07,7.866368e+07
2,KaSn-4LHo-m6vC-I4PU,240036,60064,300100,Innovexa Solutions,445470636,innovexasolutions@microfitsof.com,8.569285e+07,2024-07,72010800.0,1.368205e+07,8.569285e+07
3,Rh2k-J1o7-zndZ-cOo8,239976,59805,299781,QuantumLeap Inc.,198818316,quantumleapinc.@gemaily.net,1.713429e+08,2024-07,143985600.0,2.735726e+07,1.713429e+08
4,Vj9W-c4Pm-ja0X-fC1C,240385,59570,299955,NexaTech Industries,452680670,nexatechindustries@gemaily.net,4.862989e+07,2024-07,40865450.0,7.764436e+06,4.862989e+07


In [12]:
# Guardar el DataFrame como archivo Excel Y definir las columnas necesarias según
#lo estipulado en el correo

archivo_excel = 'resultados_comisiones.xlsx'
df_final = resultado_tab[['Fecha-Mes', 'commerce_name', 'commerce_nit', 'Valor_comision', 'Valor_iva', 'Valor_Total', 'commerce_email']]
df_final = df_final.rename(columns={'commerce_name': 'Nombre', 'commerce_nit': 'Nit', 'commerce_email': 'Correo'})
df_final.to_excel(archivo_excel, index=False)

In [13]:
df_final

,Fecha-Mes,Nombre,Nit,Valor_comision,Valor_iva,Valor_Total,Correo
0,2024-07,Zenith Corp.,28960112,29712988.5,5.645468e+06,3.535846e+07,zenithcorp.@gemaily.net
1,2024-07,FusionWave Enterprises,919341007,66103932.0,1.255975e+07,7.866368e+07,fusionwaveenterprises@microfitsof.com
2,2024-07,Innovexa Solutions,445470636,72010800.0,1.368205e+07,8.569285e+07,innovexasolutions@microfitsof.com
3,2024-07,QuantumLeap Inc.,198818316,143985600.0,2.735726e+07,1.713429e+08,quantumleapinc.@gemaily.net
4,2024-07,NexaTech Industries,452680670,40865450.0,7.764436e+06,4.862989e+07,nexatechindustries@gemaily.net


Ahora enviamos un correo para cada empresa con sus respectivos

In [14]:
calculo.enviar_correo(archivo_excel, 'danielvallejo20@outlook.com', 'danielvallejo20@outlook.com', 'xafuna20')

Error al enviar el correo: (535, b'5.7.139 Authentication unsuccessful, account locked. Contact your administrator. [BN9PR03CA0584.namprd03.prod.outlook.com 2024-09-15T15:40:22.342Z 08DCD2760C628D54]')
